# Introduction to Python's DB API
1. The Python DB API is a format that almost all relational-database-querying Python libraries use. Because of this uniform format, the overall experience of querying almost any type of relational database with Python is pretty similar, whether it's a remote PostgreSQL database or local SQLite database.
2. For those who want an even more uniform experience with a tool that can completely abstract away the differences between SQL dialects, you might try [SQLAlchemy](https://www.SQLAlchemy.org/). SQLAlchemy is extremely rich, and you could probably teach an entire class on all the things you can do with it. SQLAlchemy works especially well with Pandas (check out [pandas.read_sql_query](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)), and it has an API that replicates the standard DB API if you prefer.
3. We will be using [SQLite](https://www.sqlite.org/index.html) databases in this activity. SQLite is a lightweight DBMS for building exclusively local databases. Your computer probably has many applications with their own installations of SQLite and their own SQLite databases for storing application data. SQLite has a pretty limited syntax, but the associated package `sqlite3` comes standard with Python installations and it's a good intro to the DB API.
4. If you want to look at SQLite databases on your local machine using a nice graphical user interface, I highly recommend installing the free SQLite Browser(https://sqlitebrowser.org/). It's pretty easy to use. I've only scraped the surface of what you can do with it myself.
5. For useful notes on sqlite3, check out the [standard documentation](https://docs.python.org/3/library/sqlite3.html).
6. Finally, we will be using the [aiosqlite](https://aiosqlite.omnilib.dev/en/stable/) package to do asynchronous programming with SQLite databases and the [aiofile](https://pypi.org/project/aiofile/) package to asynchronously read files from your hard drive. To install these, call `python -m pip install aiosqlite aiofile`.

## DB API SYNTAX NOTES
1. __Don't forget to commit() your transactions and close() your database after you finish using it!__ One easy way to get confused is to write a script that inserts a bunch of stuff into the database, use the stuff that you inserted in the same transaction, close the database, and then reopen it only to discover that the stuff you inserted is nowhere to be found! If you don't call con.commit() after manipulating the database, your changes won't stick!
```py
>>> con = sqlite3.connect("my_db.sqlite")
>>> cur = con.cursor()
>>> cur.execute("INSERT INTO the_table(x,y) VALUES (1,2)")
>>> # at this point we should have used con.commit(), as we'll soon see...
>>> cur.execute("SELECT * FROM the_table WHERE x=1")
>>> x_eq1 = cur.fetchall()
>>> print(x_eq1) # everything seems to be in order...
[(1,2)]
>>> con.close() # OK, we're done here
... # time passes
>>> con = sqlite3.connect("my_db.sqlite")
>>> cur = con.cursor()
>>> cur.execute("SELECT * FROM the_table WHERE x=1")
>>> x_eq1 = cur.fetchall()
>>> print(x_eq1) # WAIT! Where did those rows I inserted go?
[]
>>> con.close()
```

2. __Don't use f-strings or similar methods for inserting variable values when working with the DB API!__ This is insecure, and can leave you open to SQL injection attacks. See https://xkcd.com/327/.
```py
>>> # Never do this -- insecure!
>>> symbol = 'RHAT'
>>> cur.execute(f"SELECT * FROM stocks WHERE symbol = '{symbol}'"
>>> # GOOD:
>>> symbol = 'RHAT'
>>> cur.execute(f"SELECT * FROM stocks WHERE symbol = ?", (symbol,))
>>> # notice that the params must be in an iterable (in this case a 1-tuple)
>>> # ALSO GOOD:
>>> vals = (1, 2)
>>> cur.execute(f"INSERT INTO the_table(x,y) VALUES (?, ?)", vals)
```

3. A handy alternative to always remembering to call con.close() at the end of a session is to use the connection in a [with statement](https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files).
Remember how you can open up text files like this:
```py
>>> with open("text_file.txt") as f:
... blah = f.read()
```
Recall that the above syntax is a good practice because Python will then automatically close the file even if there was an error.
You can do the same thing with the DB API, so that the connection is closed even if there's an error:
```py
>>> with sqlite3.connect("my_db.sqlite") as con:
... # thanks to "with" syntax, your connection will automatically close!
... cur = con.cursor()
... cur.execute("SELECT * FROM the_table WHERE x < 5")
... x_lt5 = cur.fetchall() # retrieve all the rows that were SELECTed
... cur.execute("INSERT INTO the_table(x,y) VALUES (1,2)")
... con.commit() # make sure the insertion actually sticks!
```

4. Unfortunately, the version of sqlite3 that's packaged with Python 3.6 does not support common table expressions (the `WITH temp_table_name AS (SELECT ...)` syntax). Fortunately you can still get what you want by restructuring those queries. For example,
```sql
WITH groups AS (SELECT name,
    SUM(stuff) tot_stuff
FROM tbl
GROUP BY name
)
SELECT * FROM groups LIMIT 1
```
could be replaced by
```sql
SELECT * FROM 
    (SELECT names, SUM(stuff) tot_stuff
    FROM tbl
    GROUP BY name)
LIMIT 1
```

You can use CTEs in the Python 3.9+ versions of SQLite though.

# NOTES ON ASYNCIO
1. Python has a feature called the Global Interpreter Lock (GIL) that makes it so that no single Python process can have multiple threads executing instructions in parallel. This means that you can't use Python to write the kind of multithreaded program that you could write in Java or C. (If you want to do that, you'd have to use the `multiprocessing` package).
2. However, multithreaded programs still have an important role in Python, particularly when you're dealing with slow communication-based (aka "I/O-bound") processes like web scraping or querying remote databases. While one thread is waiting for a response, the others can send requests and do data-processing tasks. Use the `threading` package if you want truly multithreaded code.
3. `asyncio` supports a form of concurrent programming that is similar to multithreaded programming. asyncio spawns __coroutines__, which are a lot like mini-threads that are all controlled by the same thread. You can't have multiple coroutines on the same thread executing at exactly the same time, but if one coroutine isn't doing anything, the others can jump in and fill the gap.
4. 
    - The main advantage of asyncio over threading is that you can easily create A LOT of coroutines; there's no real drawback to spawning dozens or maybe even hundreds of coroutines, whereas a single thread requires a lot more resources to create and manage. So if you're scraping something like 50 websites at a time, it might make sense to use asyncio rather than threading.
    - In this activity, we'll see how asyncio can be used to get reasonably good performance even with an inefficient and slow querying function (simulating a slow website connection).
5.
    - The main advantage of threading over asyncio is that, at least in Python, threading can be easier to use. asyncio requires you to pepper your code with the `async` and `await` keywords. 
    - asyncio also doesn't work with all libraries; in many cases you have to use special-built libraries to take advantage of asyncio. For example, we're using `aiosqlite` instead of `sqlite3` for the asyncio-based querying of SQLite databases, because asyncio doesn't work with sqlite3. We'll also be using `aiofile` to conurrently read text files with asyncio.
6. Finally, check out the standard documentation at https://docs.python.org/3/library/asyncio.html. *Pay special attention to when certain features were added*, because many of the functions in asyncio were added after Python 3.6.
7. Finally, there's a good explanation of how asyncio works here: https://stackoverflow.com/questions/49005651/how-does-asyncio-actually-work/51116910#51116910

## Do not include CPU-intensive code (e.g., doing a lot of arithmetic or string processing) in an asynchronous function! ##
### GOOD ###
```py
async def async_web_request(url):
    # just send the request (I/O-bound) and don't do any processing

def cpu_intensive_processing(responses):
    out = []
    for response in responses:
        a = lots_of_multiplication(response)
        out.append(lots_of_string_processing(a))
    return out

def process_all(urls):
    tasks = asyncio.gather(*[async_web_request(url)
                                for url in urls])
    loop = asyncio.get_event_loop()
    responses = loop.run_until_complete(tasks)
    loop.close()
    # now that the async part is done, do the CPU-intensive bit
    return cpu_intensive_processing(responses)
```
### BAD ###
```py
async def async_web_request(url):
    # just send the request (I/O-bound) and don't do any processing

async def process_response(url):
    response = await async_web_request(url)
    # BAD! You're doing CPU-intensive stuff in the async function
    a = lots_of_multiplication(response)
    return lots_of_string_processing(a)

def process_all(urls):
    tasks = asyncio.gather(*[process_response(url)
                                for url in urls])
    loop = asyncio.get_event_loop()
    responses = loop.run_until_complete(tasks)
    loop.close()
    return responses
```

In [15]:
# See the source file asyncio_example.py.
!python -m asyncio_example 4
# Simulate scraping the web, by creating a routine that sleeps for 1/4 second 
# before returning some data.
# We're calling it this way because asyncio doesn't play nice with 
# Jupyter notebooks.

Sending request to website 1
Sending request to website 2
Sending request to website 3

c:\Users\mjols\Documents\DS UWEC courses\730 Big Data\final\Final\p3\asyncio_example.py:21: DeprecationWarning: There is no current event loop
  all_scrapers = asyncio.gather(*[slow_io_func(x) for x in range(n)])
c:\Users\mjols\Documents\DS UWEC courses\730 Big Data\final\Final\p3\asyncio_example.py:23: DeprecationWarning: There is no current event loop
  loop = asyncio.get_event_loop()



Sending request to website 4
Got response from website 1
Got response from website 3
Got response from website 2
Got response from website 4
Processed results = [0, 1, 5, 12]
The total time required was 0.06482 seconds per website.


In [16]:
!python -m asyncio_example 12
# Notice that all 12 requests are sent out before the first response comes in.
# notice also that doing 3x as many jobs takes 1/3 as long per job, indicating
# that most of the latency in this application is just waiting for responses.

Sending request to website 1
Sending request to website 2
Sending request to website 3
Sending request to website 4
Sending request to website 5
Sending request to website 6
Sending request to website 7
Sending request to website 8
Sending request to website 9
Sending request to website 10
Sending request to website 11
Sending request to website 12
Got response from website 1
Got response from website 3
Got response from website 7
Got response from website 12
Got response from website 11
Got response from website 10

c:\Users\mjols\Documents\DS UWEC courses\730 Big Data\final\Final\p3\asyncio_example.py:21: DeprecationWarning: There is no current event loop
  all_scrapers = asyncio.gather(*[slow_io_func(x) for x in range(n)])
c:\Users\mjols\Documents\DS UWEC courses\730 Big Data\final\Final\p3\asyncio_example.py:23: DeprecationWarning: There is no current event loop
  loop = asyncio.get_event_loop()



Got response from website 9
Got response from website 6
Got response from website 8
Got response from website 5
Got response from website 2
Got response from website 4
Processed results = [0, 1, 5, 12, 22, 35, 51, 70, 92, 117, 145, 176]
The total time required was 0.02147 seconds per website.
